In [20]:
import nibabel as nib
import numpy as np

In [21]:
# Extracted information from original nifti files. 
# DeepReg throws away this information and doesn't acurately calculate mTRE

case2_affine = np.array([[ 3.33370864e-01,  1.59845248e-01,  3.33878189e-01, -3.39173775e+01],
                         [-1.82626724e-01,  4.61422384e-01, -3.86130475e-02, 4.38701019e+01],
                         [-3.21150362e-01, -9.64666754e-02,  3.68540883e-01, 6.26287537e+01],
                         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

case21_affine = np.array([[4.00647670e-01, -1.04107566e-01,  2.77501583e-01, -3.58118629e+01],
                          [ 1.67786613e-01,  4.64567333e-01, -6.78447485e-02, -8.16707916e+01],
                          [-2.44675279e-01,  1.48105368e-01,  4.07874972e-01, -8.83094406e+00],
                          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, 1.00000000e+00]])

case23_affine = np.array([[  0.31273046,   0.25455737,   0.29349992, -41.70222473],
                          [ -0.28844342,   0.40450525,  -0.04210297,   2.00646901],
                          [ -0.2584393 ,  -0.14343421,   0.40214738,   4.35490799],
                          [  0.        ,   0.        ,   0.        ,   1.        ]])

DeepReg_rescale = (128, 128, 128)

case2_size = (128, 161, 106)
case2_scale = np.array([case2_size[0] / DeepReg_rescale[0], 
                        case2_size[1] / DeepReg_rescale[1], 
                        case2_size[2] / DeepReg_rescale[2]])

case21_size = (215, 251, 191)
case21_scale = np.array([case21_size[0] / DeepReg_rescale[0], 
                         case21_size[1] / DeepReg_rescale[1], 
                         case21_size[2] / DeepReg_rescale[2]])

case23_size = (106, 147, 134)
case23_scale = np.array([case23_size[0] / DeepReg_rescale[0], 
                         case23_size[1] / DeepReg_rescale[1], 
                         case23_size[2] / DeepReg_rescale[2]])

In [22]:
def extract_centroid(image):
    """
    Extract centroid from nifti images with landmark spheres
    which have integer values according to labels
    Adapted from: https://gist.github.com/mattiaspaul/f4183f525b1cbc65e71ad23298d6436e

    :param image:
        - shape: (dim_1, dim_2, dim_3) or (batch, dim_1, dim_2, dim_3)
        - tensor or numpy array

    :return positions:
        - numpy array of labels 1
    """
    assert len(image.shape) == 3

    x = np.linspace(0, image.shape[0] - 1, image.shape[0])
    y = np.linspace(0, image.shape[1] - 1, image.shape[1])
    z = np.linspace(0, image.shape[2] - 1, image.shape[2])
    yv, xv, zv = np.meshgrid(y, x, z)
    unique = np.unique(image)[1:]  # don't include 0
    positions = np.zeros((len(unique), 3))
    for i in range(len(unique)):
        label = (image == unique[i]).astype('float32')
        xc = np.sum(label * xv) / np.sum(label)
        yc = np.sum(label * yv) / np.sum(label)
        zc = np.sum(label * zv) / np.sum(label)
        positions[i, 0] = xc
        positions[i, 1] = yc
        positions[i, 2] = zc
    return positions

In [23]:
def calculate_mTRE(xyz_true, xyz_predict):
    assert xyz_true.shape == xyz_predict.shape
    TRE = np.sqrt(np.sum(np.power(xyz_true - xyz_predict, 2), axis=1))
    mTRE = np.mean(TRE)
    return mTRE

In [101]:
def case_TREs(pred_dir, pair_number, num_labels, affine, scale):
    TREs = np.zeros(num_labels)
    for i in range(num_labels):
        label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/fixed_label.nii.gz")
        pred_label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/moving_label.nii.gz")

        label_np = label.get_fdata()
        pred_label_np = pred_label.get_fdata()
        
        label_point = nib.affines.apply_affine(affine, extract_centroid(np.round(label_np))*scale)
        pred_point = nib.affines.apply_affine(affine, extract_centroid(np.round(pred_label_np))*scale)
        
        TREs[i] = calculate_mTRE(label_point, pred_point)
        
    return TREs

# Calulating the mTRE for the 3 test cases

In [6]:
# folder path to the prediction data
prediction_dir = "logs/91_final_test/test/"

## Case 2

In [ ]:
case21_TREs = case_TREs(prediction_dir, 0, 10, case2_affine, case2_scale)
case21_TREs

In [8]:
case2_mTRE = np.mean(case2_TREs)
print(f"The mTRE for case 2 was {case2_mTRE}")

The mTRE for case 2 was 5.348180331814035


## Case 21

In [9]:
case21_TREs = case_TREs(prediction_dir, 1, 16, case21_affine, case21_scale)
case21_TREs

array([6.09022382, 3.63618927, 4.7318172 , 3.60586097, 4.58234045,
       4.40075041, 4.12417652, 4.90767851, 4.66422887, 3.25541754,
       4.75885731, 4.23768257, 3.57989204, 3.82470047, 3.86659602,
       3.74109874])

In [10]:
case21_mTRE = np.mean(case21_TREs)
print(f"The mTRE for case 21 was {case21_mTRE}")

The mTRE for case 21 was 4.250469418760417


## Case 23

In [11]:
case23_TREs = case_TREs(prediction_dir, 2, 15, case23_affine, case23_scale)
case23_TREs

array([5.89834658, 6.34542079, 7.60935995, 6.03107151, 7.04189091,
       7.65584598, 6.243868  , 7.84795153, 7.48142772, 7.44519124,
       5.05002922, 5.72686916, 5.24219633, 5.80966683, 7.79706751])

In [12]:
case23_mTRE = np.mean(case23_TREs)
print(f"The mTRE for case 23 was {case23_mTRE}")

The mTRE for case 23 was 6.615080216831815


# mTREs for CNN + SSC

In [95]:
SSC_affine_2 = np.array([[0.98227, 0.158062, 0.0692554, 4.02553e-07], 
                         [-0.19717, 0.742934, 0.0995561, 3.75881e-07],
                         [-0.0975693,  -0.0915535, 0.97393, 4.33928e-07],
                         [24.5656, 11.6949, -3.29517, 0.999908]]).T

SSC_affine_21 = np.array([[0.829324, -0.12722, -0.0146899, 29.4793], 
                         [-0.0466218, 0.93483, -0.0501401, 19.5852], 
                         [-0.202531, -0.0848386, 0.894457, 15.7477], 
                         [-7.30616e-07, -7.86814e-07, -7.86814e-07, 1.00016]])


SSC_affine_23 = np.array([[0.880435, 0.0585477, -0.0152357, 31.7375], 
                          [-0.226761, 0.881549, -0.0523897, -1.42988], 
                          [-0.0961268, -0.000586054, 0.888571, 19.7161], 
                          [9.76082e-07, 7.33881e-07, 9.04358e-07, 0.99981]])

In [ ]:
SSC_affine_2 = np.array([[0.454008, 0.116395, 0.181164, 1.03077e-06], 
-0.18823  0.51896  -0.371127  1.94371e-06
-0.0139738  0.805056  1.52682  1.32621e-06
31.9663  2.82907  33.3063  0.999684

In [5]:
def case_TREs_SSC(pred_dir, pair_number, num_labels, affine, SSC_affine, scale):
    TREs = np.zeros(num_labels)
    for i in range(num_labels):
        label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/fixed_label.nii.gz")
        pred_label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/pred_fixed_label.nii.gz")

        label_np = label.get_fdata()
        pred_label_np = pred_label.get_fdata()
        
        transformed_pred =  nib.affines.apply_affine(SSC_affine, extract_centroid(np.round(pred_label_np)))
        
        label_point = nib.affines.apply_affine(affine, extract_centroid(np.round(label_np))*scale)
        pred_point = nib.affines.apply_affine(affine, transformed_pred*scale)
        
        TREs[i] = calculate_mTRE(label_point, pred_point)
        
    return TREs

In [96]:
case2_TREs = case_TREs_SSC(prediction_dir, 0, 15, case2_affine, SSC_affine_2, case2_scale)
case2_TREs

array([ 6.89919763,  9.8596831 ,  7.00554537,  6.85536062,  9.31029165,
       11.71223314,  8.48062598,  8.93501441, 12.50851004, 11.91194414,
        8.19810729,  9.4278252 , 11.58480622, 13.08085049,  6.82395628])

In [94]:
case2_TREs = case_TREs_SSC(prediction_dir, 0, 15, case2_affine, SSC_affine_2, case2_scale)
case2_TREs

array([13.05756176, 22.03102364, 19.3258695 , 12.64887976, 20.99024809,
       22.77656643, 22.9179357 , 19.18620735, 17.63474573, 26.1468417 ,
       22.56711522, 16.98729084, 18.00435236, 16.1318155 , 12.04995052])

In [79]:
case2_TREs = case_TREs_SSC(prediction_dir, 0, 15, case2_affine, SSC_affine_2, case2_scale)
case2_TREs

array([19.24776635, 23.34605407, 21.48908862, 19.37191419, 22.78752431,
       23.63353645, 24.32054994, 21.77644473, 20.24874287, 25.51629422,
       23.65913559, 22.64904876, 20.82494708, 22.14204211, 20.5868673 ])

### Testing original alignment

In [11]:
# folder path to the prediction data
prediction_dir = "logs/91_final_test/test/"

In [24]:
test_affine_SSC_1 = np.array([[0.970629, 0.123898, 0.0611337, 4.38563e-06], 
                          [-0.187429, 0.76746, 0.160733, 3.77978e-06], 
                          [-0.164659, 0.0286062, 1.00569, 4.14213e-06], 
                          [35.0873, 6.22194, -9.00875, 0.999164]]).T

test_affine_SSC_2 = np.array([[0.454008, 0.116395, 0.181164, 1.03077e-06],
                              [-0.18823, 0.51896, -0.371127, 1.94371e-06],
                              [-0.0139738, 0.805056, 1.52682, 1.32621e-06],
                              [31.9663, 2.82907, 33.3063, 0.999684]])

test_affine_SSC_3 = np.array([[0.840965, -0.516923, -0.606431, 1.46757e-06],
                              [-0.067257, 0.0511995, -0.291972, 1.07902e-06],
                              [-0.201934, 0.657801, 0.977594, 1.70895e-07],
                              [4.65295, 78.8709, 78.4798, 0.999825]]).T

In [29]:
def case_TREs_SSC__(pred_dir, pair_number, num_labels, affine, SSC_affine, scale):
    TREs = np.zeros(num_labels)
    for i in range(num_labels):
        label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/fixed_label.nii.gz")
        pred_label = nib.load(pred_dir + f"pair_{pair_number}/label_{i}/moving_label.nii.gz")

        label_np = label.get_fdata()
        pred_label_np = pred_label.get_fdata()
        
        transformed_pred =  nib.affines.apply_affine(SSC_affine, extract_centroid(np.round(pred_label_np)))
        
        label_point = nib.affines.apply_affine(affine, extract_centroid(np.round(label_np))*scale)
        pred_point = nib.affines.apply_affine(affine, transformed_pred*scale)
        
        #label_point = extract_centroid(np.round(label_np))*scale
        #pred_point = transformed_pred*scale
        
        TREs[i] = calculate_mTRE(label_point, pred_point)
        
    return TREs

In [16]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC, case2_scale)
case2_TREs

array([16.10787796, 19.06947808, 15.31731464, 15.91987838, 19.12302418,
       21.29432886, 18.4414098 , 18.06954607, 20.56424073, 22.47567695,
       17.63872093, 20.8251736 , 20.16753683, 32.99064178, 14.55383815])

In [30]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC_1, case2_scale)
case2_TREs

array([ 8.02437875,  9.50592548,  7.63118018,  7.92777259,  9.52471048,
       10.61070698,  9.18837346,  8.99925713, 10.24778625, 11.19551575,
        8.79173582, 10.3678349 , 10.05218003, 16.41707428,  7.24569682])

In [31]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC_2, case2_scale)
case2_TREs

array([56.96751612, 70.04614542, 59.67397277, 55.93936021, 61.9529287 ,
       73.0900259 , 64.02841058, 59.97789412, 72.27584283, 70.57089491,
       64.63602626, 56.7803615 , 70.93267415, 36.24224967, 58.12097182])

In [32]:
case2_TREs = case_TREs_SSC__(prediction_dir, 0, 15, case2_affine, test_affine_SSC_3, case2_scale)
case2_TREs

array([42.02724955, 17.2086385 , 20.22055004, 47.09045049, 17.20753374,
       18.34937172, 17.2592487 , 22.92149937, 31.99936864, 25.17083639,
       15.87855536, 36.59913957, 29.20965219, 39.01910603, 53.18504964])

## ####

In [3]:
from scipy.ndimage import zoom
prediction_dir = "logs/91_final_test/test/"

In [4]:
fixed_image = nib.load(prediction_dir + "pair_0/fixed_image.nii.gz").get_fdata()
moving_image = nib.load(prediction_dir + "pair_0/moving_image.nii.gz").get_fdata()
pred_fixed_image = nib.load(prediction_dir + "pair_0/pred_fixed_image.nii.gz").get_fdata()

In [ ]:
unscaled_fixed_image = zoom(fixed_image, case2_size, order = 1)

In [ ]:
fixed_image = nib.load(prediction_dir + "pair_0/fixed_image.nii.gz").get_fdata()
moving_image = nib.load(prediction_dir + "pair_0/moving_image.nii.gz").get_fdata()
pred_fixed_image = nib.load(prediction_dir + "pair_0/pred_fixed_image.nii.gz").get_fdata()

unscaled_fixed_image = zoom(fixed_image, case2_size)
unscaled_moving_image = zoom(moving_image, case2_size)
unscaled_pred_fixed_image = zoom(pred_fixed_image, case2_size)

unscaled_fixed_image_nifti = nib.Nifti1Image(unscaled_fixed_image, case2_affine)
unscaled_moving_image_nifti = nib.Nifti1Image(unscaled_moving_image, case2_affine)
unscaled_pred_fixed_image_nifti = nib.Nifti1Image(pred_unscaled_fixed_image, case2_affine)

unscaled_fixed_image_nifti.to_filename(prediction_dir + "pair_0/Unscaled/unscaled_fixed_image.nii.gz")
unscaled_moving_image_nifti.to_filename(prediction_dir + "pair_0/Unscaled/unscaled_moving_image.nii.gz")
unscaled_pred_fixed_image_nifti.to_filename(prediction_dir + "pair_0/Unscaled/unscaled_pred_fixed_image.nii.gz")

In [ ]:
fixed_image = nib.load(prediction_dir + "pair_1/fixed_image.nii.gz").get_fdata()
moving_image = nib.load(prediction_dir + "pair_1/moving_image.nii.gz").get_fdata()
pred_fixed_image = nib.load(prediction_dir + "pair_1/pred_fixed_image.nii.gz").get_fdata()

unscaled_fixed_image = zoom(fixed_image, case21_size)
unscaled_moving_image = zoom(moving_image, case21_size)
unscaled_pred_fixed_image = zoom(pred_fixed_image, case21_size)

unscaled_fixed_image_nifti = nib.Nifti1Image(unscaled_fixed_image, case21_affine)
unscaled_moving_image_nifti = nib.Nifti1Image(unscaled_moving_image, case21_affine)
unscaled_pred_fixed_image_nifti = nib.Nifti1Image(pred_unscaled_fixed_image, case21_affine)

unscaled_fixed_image_nifti.to_filename(prediction_dir + "pair_1/Unscaled/unscaled_fixed_image.nii.gz")
unscaled_moving_image_nifti.to_filename(prediction_dir + "pair_1/Unscaled/unscaled_moving_image.nii.gz")
unscaled_pred_fixed_image_nifti.to_filename(prediction_dir + "pair_1/Unscaled/unscaled_pred_fixed_image.nii.gz")

In [ ]:
fixed_image = nib.load(prediction_dir + "pair_2/fixed_image.nii.gz").get_fdata()
moving_image = nib.load(prediction_dir + "pair_2/moving_image.nii.gz").get_fdata()
pred_fixed_image = nib.load(prediction_dir + "pair_2/pred_fixed_image.nii.gz").get_fdata()

unscaled_fixed_image = zoom(fixed_image, case23_size)
unscaled_moving_image = zoom(moving_image, case23_size)
unscaled_pred_fixed_image = zoom(pred_fixed_image, case23_size)

unscaled_fixed_image_nifti = nib.Nifti1Image(unscaled_fixed_image, case23_affine)
unscaled_moving_image_nifti = nib.Nifti1Image(unscaled_moving_image, case23_affine)
unscaled_pred_fixed_image_nifti = nib.Nifti1Image(pred_unscaled_fixed_image, case23_affine)

unscaled_fixed_image_nifti.to_filename(prediction_dir + "pair_2/Unscaled/unscaled_fixed_image.nii.gz")
unscaled_moving_image_nifti.to_filename(prediction_dir + "pair_2/Unscaled/unscaled_moving_image.nii.gz")
unscaled_pred_fixed_image_nifti.to_filename(prediction_dir + "pair_2/Unscaled/unscaled_pred_fixed_image.nii.gz")

In [ ]:
case_size

In [106]:
case2_size

(128, 161, 106)

In [110]:
fixed_image.shape

(128, 128, 128)

In [111]:
case2_size

(128, 161, 106)

In [113]:
cv2.resize(fixed_image, (128,161,106))

TypeError: function takes exactly 2 arguments (3 given)

In [114]:
fixed_image

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.